# Phase 1 - Initial Exploration

Goals:
- Load data
- Gather basic info (shape, column names, data types, etc.)
- Generate summary statistics
- Identify and understand distribution of key variables
- Establish 3 questions to answer w/ data

Deliverables/Outcomes:
- Quarto slidedeck (5-10 slides) summarizing findings

## Load Data

We are working with the Detections of Highly Pathogenic Avian Influenza (HPAI) in Wild Birds dataset from the USDA. This tabular CSV dataset contains data regarding confirmed HPAI cases for birds in the US. Data goes back to 2022 and is regularly updated.

In [ ]:
import pandas as pd

dataset_path = '../data/HPAI Detections in Wild Birds.csv'
hpai_data = pd.read_csv(dataset_path)

print(hpai_data.head(n=5)) # view the first 5 rows of the dataset


          State    County Collection Date Date Detected HPAI Strain  \
0  North Dakota      Cass       9/12/2025     9/19/2025       EA H5   
1  Pennsylvania     Bucks        9/8/2025     9/19/2025       EA H5   
2  Pennsylvania  Delaware        9/4/2025     9/19/2025       EA H5   
3    New Jersey    Warren       9/11/2025     9/19/2025       EA H5   
4    New Jersey    Warren       9/11/2025     9/19/2025       EA H5   

    Bird Species WOAH Classification      Sampling Method   Submitting Agency  
0   Canada goose           Wild bird  Morbidity/Mortality    ND Game and Fish  
1  Black vulture           Wild bird  Morbidity/Mortality  PA Game Commission  
2  Black vulture           Wild bird  Morbidity/Mortality  PA Game Commission  
3  Black vulture           Wild bird  Morbidity/Mortality              NJ DEP  
4  Black vulture           Wild bird  Morbidity/Mortality              NJ DEP  


## Gather Basic Info

- Shape (number of rows and columns)
- Column names and data types
- Possible values and their meanings